In [36]:
import pandas as pd
import numpy as np

In [86]:
na_values = ['-9999','9999']
df = pd.read_csv('BZN-weather-data_4.1895-6.2015.csv',
                        na_values=na_values,
                        index_col='DATE',
                        parse_dates=True)

df.head()

,STATION,STATION_NAME,ELEVATION,LATITUDE,LONGITUDE,MDPR,Measurement Flag,Quality Flag,Source Flag,Time of Observation,...,WT18,Measurement Flag.16,Quality Flag.16,Source Flag.16,Time of Observation.16,WT03,Measurement Flag.17,Quality Flag.17,Source Flag.17,Time of Observation.17
DATE,,,,,,,,,,,,,,,,,,,,,
1892-04-08,GHCND:USC00241044,BOZEMAN MONTANA SU MT US,unknown,unknown,unknown,NaN,,,,NaN,...,NaN,,,,NaN,NaN,,,,NaN
1892-04-09,GHCND:USC00241044,BOZEMAN MONTANA SU MT US,unknown,unknown,unknown,NaN,,,,NaN,...,NaN,,,,NaN,NaN,,,,NaN
1892-04-10,GHCND:USC00241044,BOZEMAN MONTANA SU MT US,unknown,unknown,unknown,NaN,,,,NaN,...,NaN,,,,NaN,NaN,,,,NaN
1892-04-11,GHCND:USC00241044,BOZEMAN MONTANA SU MT US,unknown,unknown,unknown,NaN,,,,NaN,...,NaN,,,,NaN,NaN,,,,NaN
1892-04-12,GHCND:USC00241044,BOZEMAN MONTANA SU MT US,unknown,unknown,unknown,NaN,,,,NaN,...,NaN,,,,NaN,NaN,,,,NaN


In [101]:
# Data cleaning beyond import? Doesn't seem necessary

In [108]:
# Making output dataframe w/ columns I really care about
output_df = pd.DataFrame(df[['PRCP','TMAX','TMIN']])

output_df.head()

,PRCP,TMAX,TMIN
DATE,,,
1892-04-08,0,78,-150
1892-04-09,0,128,6
1892-04-10,0,100,6
1892-04-11,0,89,-6
1892-04-12,0,94,-28


In [141]:
# Calculating historical extremes/averages by month and day

# Hoping I can get away with ignoring leap years

# df[df.index.month == 12] # How to select rows by month
# df[df.index.day === 2] # How to select rows by day

daysMonth = { 1:31, 2:29, 3:31, 4:30, 5:31, 6:30,
             7:31, 8:31, 9:30, 10:31, 11:30, 12:31}

avgHigh = {}
avgLow = {}
recHigh = {}
recLow = {}

for month in range(1, 13):
    for day in range(1, daysMonth[month]+1):
        
        tMaxList = df[(df.index.month == month) & (df.index.day == day)]['TMAX']
        tMinList = df[(df.index.month == month) & (df.index.day == day)]['TMIN']
        # prcpList = df[(df.index.month == month) & (df.index.day == day)]['PRCP']
        
        # Calculate values of interest an assign them to a dict
        # hashed with a month/day tuple
        avgHigh[(month, day)] = tMaxList.mean()
        avgLow[(month, day)] = tMinList.mean()
        recHigh[(month, day)] = tMaxList.max()
        recLow[(month, day)] = tMinList.min()
        
        # print "%r/%r" % (month, day)
        #  print "Avg tMax: %d; Avg tMin: %d; HistHigh: %d, HistLow: %d" % (tMaxAvg, tMinAvg, tHistHigh, tHistLow)
        


In [142]:
# Adding columns to output_df

output_df['AVGHIGH'] = np.nan
output_df['AVGLOW'] = np.nan
output_df['RECHIGH'] = np.nan
output_df['RECLOW'] = np.nan

for index, row in output_df.iterrows():
    output_df['AVGHIGH'][index] = avgHigh[(index.month, index.day)]
    output_df['AVGLOW'][index] = avgLow[(index.month, index.day)]
    output_df['RECHIGH'][index] = recHigh[(index.month, index.day)]
    output_df['RECLOW'][index] = recLow[(index.month, index.day)]

In [143]:
# Preview output
output_df[output_df.index.year == 2014]

,PRCP,TMAX,TMIN,AVGHIGH,AVGLOW,RECHIGH,RECLOW
DATE,,,,,,,
2014-01-01,5,33,-17,-16.925620,-120.793388,139,-333
2014-01-02,0,28,-83,-6.809917,-120.355372,117,-311
2014-01-03,5,67,-11,-5.454545,-114.727273,117,-311
2014-01-04,10,11,-133,-4.123967,-113.933884,111,-317
2014-01-05,18,-72,-194,-0.429752,-111.041322,106,-339
2014-01-06,0,-61,-239,-4.416667,-111.000000,128,-344
2014-01-07,8,17,-78,-1.115702,-104.876033,133,-361
2014-01-08,5,33,-61,3.115702,-106.487603,183,-361
2014-01-09,10,33,-17,-3.049587,-107.363636,128,-328


In [144]:
output_df[output_df.index.year == 2014].to_csv("BZN-weather-data_2014.csv")

In [148]:
output_df.to_csv("BZN-weather-data_entire.csv")